# HW08. Алгоритм Леска 

In [1]:
from nltk.corpus import wordnet as wn
import nltk
from collections import Counter

In [2]:
def count_overlap(sentence_a, sentence_b):
    result = sum((Counter(sentence_a) & Counter(sentence_b)).values())
    return result

In [3]:
def lesk( word, sentence ):
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)
    
    for i, syns in enumerate(synsets):
        definition = syns.definition()
        definition = definition.lower().split()
        overlap = count_overlap(definition, sentence)
        
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
             
    return bestsense

In [4]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time'.split()) # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

In [5]:
wn.synsets('day')

[Synset('day.n.01'),
 Synset('day.n.02'),
 Synset('day.n.03'),
 Synset('day.n.04'),
 Synset('day.n.05'),
 Synset('day.n.06'),
 Synset('day.n.07'),
 Synset('sidereal_day.n.01'),
 Synset('day.n.09'),
 Synset('day.n.10')]

In [6]:
# с помощью этого индекса достаем нужный синсет
wn.synsets('day')[1].definition()

'some point or period in time'

Проверьте насколько хорошо работает такой метод на реальном датасете. http://lcl.uniroma1.it/wsdeval/ - большой фреймворк для оценки WSD. Там много данных и я взял кусочек, чтобы не было проблем с памятью

In [7]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [8]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

In [9]:
wn.lemma_from_key('review%2:31:00::').synset()

Synset('review.v.01')

In [10]:
i = lesk('long', "It was a long day")
wn.synsets('long')[i]

Synset('long.a.01')

In [11]:
# тут подходит второй синсет (!в вашем решении вместо индекса единицы будет результат работы функции lesk)
wn.synsets('long')[1] == wn.lemma_from_key('long%3:00:02::').synset()

True

In [12]:
# можно сравнить со всеми, чтобы убедиться
[(syns, wn.synsets('long')[i] == wn.lemma_from_key('long%3:00:02::').synset()) 
  for i, syns in enumerate(wn.synsets('long'))]

[(Synset('hanker.v.01'), False),
 (Synset('long.a.01'), True),
 (Synset('long.a.02'), False),
 (Synset('long.s.03'), False),
 (Synset('retentive.a.01'), False),
 (Synset('long.a.05'), False),
 (Synset('long.a.06'), False),
 (Synset('long.s.07'), False),
 (Synset('farseeing.s.02'), False),
 (Synset('long.s.09'), False),
 (Synset('long.r.01'), False),
 (Synset('long.r.02'), False)]

#### Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)


In [13]:
true = 0
total = 0

for sent in corpus_wsd[:1000]:
    for sense, lemma, word in sent:
        if sense:
            total += 1
            context = " ".join([s[2] for s in sent])
            ind = lesk(lemma, context)
            try:
                if wn.synsets(lemma)[ind] == wn.lemma_from_key(sense).synset():
                    true += 1
            except:
                pass

print('accuracy:', (true/total))

accuracy: 0.35574261992619927
